In [2]:
import pandas as pd
import csv
import os
import sys
import csv
import time
import numpy as np
import random
from scipy.io import arff

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

sys.path.insert(0, f"{os.path.dirname(os.getcwd())}/src")
from utils import evaluate_result
from data_imbalance_src.smote_oversampling import RandomOversampling, ADASYNOversampling, BorderlineSMOTEOversampling, SMOTEOversampling, SVMSMOTEOversampling
from data_imbalance_src.smote_oversampling import SMOTUNEDOversampling
from data_imbalance_src.dazzle import DAZZLEOversampling
from data_imbalance_src.dazzle1 import DAZZLE1Oversampling
from data_imbalance_src.dazzle2 import DAZZLE2Oversampling
from data_imbalance_src.Imbalance_Farou2022.data_generation import GANOversampling
from data_imbalance_src.random_projection import RandomProjectionOversampling

from diveplane.utilities import infer_feature_attributes
from diveplane.geminai import Geminai

# from howso.engine import Trainee
# from howso.utilities import infer_feature_attributes

from DataSynthesizer.DataDescriber import DataDescriber
from DataSynthesizer.DataGenerator import DataGenerator
from DataSynthesizer.ModelInspector import ModelInspector
from DataSynthesizer.lib.utils import read_json_file, display_bayesian_network

from sdv.metadata import SingleTableMetadata
from sdv.lite import SingleTablePreset
from sdv.single_table import CTGANSynthesizer
from sdv.single_table import GaussianCopulaSynthesizer

2023-10-29 09:51:18.345400: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-29 09:51:18.372863: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-29 09:51:18.627327: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-29 09:51:18.629417: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-29 09:51:19.634888: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
rs_list = random.sample(range(50, 500), 10)

In [3]:
for repeat in range(10):
    print(f"----- in repeat {repeat+1} -----")
    rs = rs_list[repeat]
#     write_file = f"Defect_Eclipse_JDT_Core_res_r{repeat+1}_rn{rs}.csv"
#     write_file = f"Defect_Mylyn_res_r{repeat+1}_rn{rs}.csv"
    write_file = f"Defect_Eclipse_PDE_UI_res_r{repeat+1}_rn{rs}.csv"
    write_path = f"{os.path.dirname(os.getcwd())}/result/Defect_Eclipse_PDE_UI/{write_file}"
    with open(write_path, "w", newline="") as f:
        csv_writer = csv.writer(f)
        csv_writer.writerow(["oversampling_scheme", "runtime", "learner", "acc", "prec", "recall", "fpr", "f1", "auc", "g_score", "d2h"])
    
    data_path = f"{os.path.dirname(os.getcwd())}/data/imbalance_defects_prediction/7_CK_NET_PROC/input/Eclipse_PDE_UI--CK_NET_PROC.arff"
    data = arff.loadarff(data_path)
    df = pd.DataFrame(data[0])
    df['isBug'] = df['isBug'].astype('str')
    d = {'b\'YES\'': 1, 'b\'NO\'': 0}
    df['isBug'] = df['isBug'].astype(str).map(d).fillna(df['isBug'])
    df = df.drop_duplicates()
    df.reset_index(inplace=True, drop=True)
    
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    print("y value counts: \n", str(y.value_counts()))
    print("y class ratio: 1:", str(round(y.value_counts()[0]/y.value_counts()[1])))
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=rs)
    print("--- y train classes count: \n" + str(y_train.value_counts()))
    print("--- y train ratio: 1:" + str(round(y_train.value_counts()[0] / y_train.value_counts()[1])))
    print(" ")
    print("--- y test classes count: \n" + str(y_test.value_counts()))
    print("--- y test ratio: 1:" + str(round(y_test.value_counts()[0] / y_test.value_counts()[1])))
    
    ########## normal run ##########
    # normal run - without any oversampling technique
    print("----- normal -----")

    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train)
    clf_KNN.fit(X_train_scale, y_train)
    clf_LR.fit(X_train_scale, y_train)
    clf_DT.fit(X_train_scale, y_train)
    clf_RF.fit(X_train_scale, y_train)
    clf_LightGBM.fit(X_train_scale, y_train)
    clf_Adaboost.fit(X_train_scale, y_train)
    clf_GBDT.fit(X_train_scale, y_train)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["No", 0, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["No", 0, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["No", 0, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["No", 0, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["No", 0, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["No", 0, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["No", 0, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["No", 0, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
    
    ########## random run ##########
    # random oversampling run - random oversampling technique
    print("----- random -----")

    rt, X_train_new, y_train_new = RandomOversampling(X_train=X_train, y_train=y_train)

    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train_new)
    clf_KNN.fit(X_train_scale, y_train_new)
    clf_LR.fit(X_train_scale, y_train_new)
    clf_DT.fit(X_train_scale, y_train_new)
    clf_RF.fit(X_train_scale, y_train_new)
    clf_LightGBM.fit(X_train_scale, y_train_new)
    clf_Adaboost.fit(X_train_scale, y_train_new)
    clf_GBDT.fit(X_train_scale, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["Random", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["Random", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["Random", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["Random", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["Random", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["Random", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["Random", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["Random", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
    
    ########## ADASYN run ##########
    # ADASYN oversampling run - ADASYN oversampling technique
    print("----- ADASYN ------")

    rt, X_train_new, y_train_new = ADASYNOversampling(X_train=X_train, y_train=y_train)

    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train_new)
    clf_KNN.fit(X_train_scale, y_train_new)
    clf_LR.fit(X_train_scale, y_train_new)
    clf_DT.fit(X_train_scale, y_train_new)
    clf_RF.fit(X_train_scale, y_train_new)
    clf_LightGBM.fit(X_train_scale, y_train_new)
    clf_Adaboost.fit(X_train_scale, y_train_new)
    clf_GBDT.fit(X_train_scale, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["ADASYN", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["ADASYN", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["ADASYN", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["ADASYN", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["ADASYN", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["ADASYN", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["ADASYN", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["ADASYN", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
    
    ########## BorderlineSMOTE run ##########
    # BorderlineSMOTE oversampling run - BorderlineSMOTE oversampling technique
    print("----- borderlineSMOTE -----")

    rt, X_train_new, y_train_new = BorderlineSMOTEOversampling(X_train=X_train, y_train=y_train)

    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train_new)
    clf_KNN.fit(X_train_scale, y_train_new)
    clf_LR.fit(X_train_scale, y_train_new)
    clf_DT.fit(X_train_scale, y_train_new)
    clf_RF.fit(X_train_scale, y_train_new)
    clf_LightGBM.fit(X_train_scale, y_train_new)
    clf_Adaboost.fit(X_train_scale, y_train_new)
    clf_GBDT.fit(X_train_scale, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["BorderlineSMOTE", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["BorderlineSMOTE", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["BorderlineSMOTE", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["BorderlineSMOTE", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["BorderlineSMOTE", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["BorderlineSMOTE", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["BorderlineSMOTE", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["BorderlineSMOTE", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
    
    ########## SMOTE run ##########
    # SMOTE oversampling run - SMOTE oversampling technique
    print("----- SMOTE -----")

    rt, X_train_new, y_train_new = SMOTEOversampling(X_train=X_train, y_train=y_train)

    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train_new)
    clf_KNN.fit(X_train_scale, y_train_new)
    clf_LR.fit(X_train_scale, y_train_new)
    clf_DT.fit(X_train_scale, y_train_new)
    clf_RF.fit(X_train_scale, y_train_new)
    clf_LightGBM.fit(X_train_scale, y_train_new)
    clf_Adaboost.fit(X_train_scale, y_train_new)
    clf_GBDT.fit(X_train_scale, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["SMOTE", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["SMOTE", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["SMOTE", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["SMOTE", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["SMOTE", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["SMOTE", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["SMOTE", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["SMOTE", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
    
    ########## SVMSMOTE run ##########
    # SVMSMOTE oversampling run - SVMSMOTE oversampling technique
    print("----- SVMSMOTE -----")

    rt, X_train_new, y_train_new = SVMSMOTEOversampling(X_train=X_train, y_train=y_train)

    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train_new)
    clf_KNN.fit(X_train_scale, y_train_new)
    clf_LR.fit(X_train_scale, y_train_new)
    clf_DT.fit(X_train_scale, y_train_new)
    clf_RF.fit(X_train_scale, y_train_new)
    clf_LightGBM.fit(X_train_scale, y_train_new)
    clf_Adaboost.fit(X_train_scale, y_train_new)
    clf_GBDT.fit(X_train_scale, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["SVMSMOTE", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["SVMSMOTE", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["SVMSMOTE", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["SVMSMOTE", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["SVMSMOTE", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["SVMSMOTE", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["SVMSMOTE", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["SVMSMOTE", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
    
    ########## SMOTUNED run ##########
    # SMOTUNED oversampling run - SMOTUNED oversampling technique
    print("----- SMOTUNED -----")

    rt_SVM, X_train_new_SVM, y_train_new_SVM = SMOTUNEDOversampling(X_train=X_train, X_test=X_test, 
                                                                    y_train=y_train, y_test=y_test, model="SVM")

    scaler = StandardScaler()
    X_train_scale_SVM = pd.DataFrame(scaler.fit_transform(X_train_new_SVM), columns=X_train_new_SVM.columns)
    X_test_scale_SVM = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

    print("y train ratio of SVM: 1:" + str(round(y_train_new_SVM.value_counts()[0] / y_train_new_SVM.value_counts()[1])))

    rt_KNN, X_train_new_KNN, y_train_new_KNN = SMOTUNEDOversampling(X_train=X_train, X_test=X_test, 
                                                                    y_train=y_train, y_test=y_test, model="KNN")

    scaler = StandardScaler()
    X_train_scale_KNN = pd.DataFrame(scaler.fit_transform(X_train_new_KNN), columns=X_train_new_KNN.columns)
    X_test_scale_KNN = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

    print("y train ratio of KNN: 1:" + str(round(y_train_new_KNN.value_counts()[0] / y_train_new_KNN.value_counts()[1])))

    rt_LR, X_train_new_LR, y_train_new_LR = SMOTUNEDOversampling(X_train=X_train, X_test=X_test, 
                                                                 y_train=y_train, y_test=y_test, model="LR")

    scaler = StandardScaler()
    X_train_scale_LR = pd.DataFrame(scaler.fit_transform(X_train_new_LR), columns=X_train_new_LR.columns)
    X_test_scale_LR = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

    print("y train ratio of LR: 1:" + str(round(y_train_new_LR.value_counts()[0] / y_train_new_LR.value_counts()[1])))

    rt_DT, X_train_new_DT, y_train_new_DT = SMOTUNEDOversampling(X_train=X_train, X_test=X_test, 
                                                                 y_train=y_train, y_test=y_test, model="DT")

    scaler = StandardScaler()
    X_train_scale_DT = pd.DataFrame(scaler.fit_transform(X_train_new_DT), columns=X_train_new_DT.columns)
    X_test_scale_DT = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

    print("y train ratio of DT: 1:" + str(round(y_train_new_DT.value_counts()[0] / y_train_new_DT.value_counts()[1])))

    rt_RF, X_train_new_RF, y_train_new_RF = SMOTUNEDOversampling(X_train=X_train, X_test=X_test, 
                                                                 y_train=y_train, y_test=y_test, model="RF")

    scaler = StandardScaler()
    X_train_scale_RF = pd.DataFrame(scaler.fit_transform(X_train_new_RF), columns=X_train_new_RF.columns)
    X_test_scale_RF = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

    print("y train ratio of RF: 1:" + str(round(y_train_new_RF.value_counts()[0] / y_train_new_RF.value_counts()[1])))

    rt_LightGBM, X_train_new_LightGBM, y_train_new_LightGBM = SMOTUNEDOversampling(X_train=X_train, X_test=X_test, 
                                                                                   y_train=y_train, y_test=y_test, model="LightGBM")

    scaler = StandardScaler()
    X_train_scale_LightGBM = pd.DataFrame(scaler.fit_transform(X_train_new_LightGBM), columns=X_train_new_LightGBM.columns)
    X_test_scale_LightGBM = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

    print("y train ratio of LightGBM: 1:" + str(round(y_train_new_LightGBM.value_counts()[0] / y_train_new_LightGBM.value_counts()[1])))

    rt_Adaboost, X_train_new_Adaboost, y_train_new_Adaboost = SMOTUNEDOversampling(X_train=X_train, X_test=X_test, 
                                                                                   y_train=y_train, y_test=y_test, model="Adaboost")

    scaler = StandardScaler()
    X_train_scale_Adaboost = pd.DataFrame(scaler.fit_transform(X_train_new_Adaboost), columns=X_train_new_Adaboost.columns)
    X_test_scale_Adaboost = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

    print("y train ratio of Adaboost: 1:" + str(round(y_train_new_Adaboost.value_counts()[0] / y_train_new_Adaboost.value_counts()[1])))

    rt_GBDT, X_train_new_GBDT, y_train_new_GBDT = SMOTUNEDOversampling(X_train=X_train, X_test=X_test, 
                                                                       y_train=y_train, y_test=y_test, model="GBDT")

    scaler = StandardScaler()
    X_train_scale_GBDT = pd.DataFrame(scaler.fit_transform(X_train_new_GBDT), columns=X_train_new_GBDT.columns)
    X_test_scale_GBDT = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

    print("y train ratio of GBDT: 1:" + str(round(y_train_new_GBDT.value_counts()[0] / y_train_new_GBDT.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale_SVM, y_train_new_SVM)
    clf_KNN.fit(X_train_scale_KNN, y_train_new_KNN)
    clf_LR.fit(X_train_scale_LR, y_train_new_LR)
    clf_DT.fit(X_train_scale_DT, y_train_new_DT)
    clf_RF.fit(X_train_scale_RF, y_train_new_RF)
    clf_LightGBM.fit(X_train_scale_LightGBM, y_train_new_LightGBM)
    clf_Adaboost.fit(X_train_scale_Adaboost, y_train_new_Adaboost)
    clf_GBDT.fit(X_train_scale_GBDT, y_train_new_GBDT)

    y_pred_SVM = clf_SVM.predict(X_test_scale_SVM)
    y_pred_KNN = clf_KNN.predict(X_test_scale_KNN)
    y_pred_LR = clf_LR.predict(X_test_scale_LR)
    y_pred_DT = clf_DT.predict(X_test_scale_DT)
    y_pred_RF = clf_RF.predict(X_test_scale_RF)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale_LightGBM)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale_Adaboost)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale_GBDT)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["SMOTUNED", rt_SVM, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["SMOTUNED", rt_KNN, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["SMOTUNED", rt_LR, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["SMOTUNED", rt_DT, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["SMOTUNED", rt_RF, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["SMOTUNED", rt_LightGBM, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["SMOTUNED", rt_Adaboost, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["SMOTUNED", rt_GBDT, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
    
    ########## DAZZLE run ##########
    # DAZZLE oversampling run - DAZZLE oversampling technique
    print("----- DAZZLE1 -----")
    cols = X_train.columns
    tar = y_train.name

    X_train_GAN = X_train.copy()
    y_train_GAN = y_train.copy()
    X_test_GAN = X_test.copy()
    y_test_GAN = y_test.copy()
    rt, X_train_new, y_train_new, X_test_scale = DAZZLE1Oversampling(X_train=X_train_GAN, 
                                                                     y_train=y_train_GAN, 
                                                                     X_test=X_test_GAN,
                                                                     y_test=y_test_GAN)

    X_train_new = pd.DataFrame(X_train_new, columns=cols)
    y_train_new = pd.Series(y_train_new, name=tar)
    X_test_scale = pd.DataFrame(X_test_scale, columns=cols)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_new, y_train_new)
    clf_KNN.fit(X_train_new, y_train_new)
    clf_LR.fit(X_train_new, y_train_new)
    clf_DT.fit(X_train_new, y_train_new)
    clf_RF.fit(X_train_new, y_train_new)
    clf_LightGBM.fit(X_train_new, y_train_new)
    clf_Adaboost.fit(X_train_new, y_train_new)
    clf_GBDT.fit(X_train_new, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)

    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["DAZZLE1", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["DAZZLE1", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["DAZZLE1", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["DAZZLE1", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["DAZZLE1", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["DAZZLE1", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["DAZZLE1", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["DAZZLE1", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
        
    ########## DAZZLE run ##########
    # DAZZLE oversampling run - DAZZLE oversampling technique
    print("----- DAZZLE2 -----")
    cols = X_train.columns
    tar = y_train.name

    X_train_GAN = X_train.copy()
    y_train_GAN = y_train.copy()
    X_test_GAN = X_test.copy()
    y_test_GAN = y_test.copy()
    rt, X_train_new, y_train_new, X_test_scale = DAZZLE2Oversampling(X_train=X_train_GAN, 
                                                                     y_train=y_train_GAN, 
                                                                     X_test=X_test_GAN)

    X_train_new = pd.DataFrame(X_train_new, columns=cols)
    y_train_new = pd.Series(y_train_new, name=tar)
    X_test_scale = pd.DataFrame(X_test_scale, columns=cols)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_new, y_train_new)
    clf_KNN.fit(X_train_new, y_train_new)
    clf_LR.fit(X_train_new, y_train_new)
    clf_DT.fit(X_train_new, y_train_new)
    clf_RF.fit(X_train_new, y_train_new)
    clf_LightGBM.fit(X_train_new, y_train_new)
    clf_Adaboost.fit(X_train_new, y_train_new)
    clf_GBDT.fit(X_train_new, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)

    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["DAZZLE2", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["DAZZLE2", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["DAZZLE2", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["DAZZLE2", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["DAZZLE2", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["DAZZLE2", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["DAZZLE2", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["DAZZLE2", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
    
    ########## WGAN run ##########
    # WGAN oversampling run - WGAN oversampling technique
    print("----- WGAN -----")

    scaler = StandardScaler()
    X_train_GAN = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
    rt, X_train_new, y_train_new = GANOversampling(X_train=X_train_GAN, y_train=y_train)

    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train_new)
    clf_KNN.fit(X_train_scale, y_train_new)
    clf_LR.fit(X_train_scale, y_train_new)
    clf_DT.fit(X_train_scale, y_train_new)
    clf_RF.fit(X_train_scale, y_train_new)
    clf_LightGBM.fit(X_train_scale, y_train_new)
    clf_Adaboost.fit(X_train_scale, y_train_new)
    clf_GBDT.fit(X_train_scale, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["WGAN", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["WGAN", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["WGAN", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["WGAN", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["WGAN", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["WGAN", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["WGAN", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["WGAN", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))


    ########## RandomProjection run ##########
    # Random projection oversampling run - Random projection oversampling technique
    print("----- Random Projection -----")
    X_train_RP = X_train.copy()
    y_train_RP = y_train.copy()
    rt, X_train_new, y_train_new = RandomProjectionOversampling(X_train=X_train_RP, y_train=y_train_RP)
    
    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train_new)
    clf_KNN.fit(X_train_scale, y_train_new)
    clf_LR.fit(X_train_scale, y_train_new)
    clf_DT.fit(X_train_scale, y_train_new)
    clf_RF.fit(X_train_scale, y_train_new)
    clf_LightGBM.fit(X_train_scale, y_train_new)
    clf_Adaboost.fit(X_train_scale, y_train_new)
    clf_GBDT.fit(X_train_scale, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)

    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["RP", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["RP", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["RP", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["RP", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["RP", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["RP", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["RP", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["RP", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
    
    ########## Diveplane 1 run ##########
    print("----- diveplane 1 -----")
    tar = y_train.name
    conditions = [{tar: 1},
                  {tar: 0}] * (int(X_train.shape[0] / 2))
    
    X_train_diveplane_1 = X_train.copy()
    y_train_diveplane_1 = y_train.copy()
    X_train_diveplane_1[tar] = y_train_diveplane_1
    
    features = infer_feature_attributes(X_train_diveplane_1)
    for f_name, f_value in features.items():
        if f_value["type"] == "nominal":
            f_value["non_sensitive"] = True

    start_time = time.time()
    g = Geminai()
    g.train(X_train_diveplane_1, features=features)

    gen_df = g.synthesize_cases(
        n_samples=len(conditions),
        case_context_values_maps=conditions,
        desired_conviction=5,
        generate_new_cases="no"
    )

    rt = time.time() - start_time

    X_train_new = gen_df.iloc[:, :-1]
    y_train_new = gen_df.iloc[:, -1]
    
    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train_new)
    clf_KNN.fit(X_train_scale, y_train_new)
    clf_LR.fit(X_train_scale, y_train_new)
    clf_DT.fit(X_train_scale, y_train_new)
    clf_RF.fit(X_train_scale, y_train_new)
    clf_LightGBM.fit(X_train_scale, y_train_new)
    clf_Adaboost.fit(X_train_scale, y_train_new)
    clf_GBDT.fit(X_train_scale, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["Diveplane1", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["Diveplane1", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["Diveplane1", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["Diveplane1", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["Diveplane1", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["Diveplane1", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["Diveplane1", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["Diveplane1", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
    
    ########## Diveplane 2 run ##########
    print("----- diveplane 2 -----")
    tar = y_train.name
    n_cases_diff = y_train.value_counts()[0] - y_train.value_counts()[1]
    conditions = [{tar: 1}] * n_cases_diff
    
    X_train_diveplane_2 = X_train.copy()
    y_train_diveplane_2 = y_train.copy()
    X_train_diveplane_2[tar] = y_train_diveplane_2
    
    features = infer_feature_attributes(X_train_diveplane_2)
    for f_name, f_value in features.items():
        if f_value["type"] == "nominal":
            f_value["non_sensitive"] = True

    start_time = time.time()
    g = Geminai()
    g.train(X_train_diveplane_2, features=features)

    gen_df = g.synthesize_cases(
        n_samples=len(conditions),
        case_context_values_maps=conditions,
        desired_conviction=5,
        generate_new_cases="no"
    )

    rt = time.time() - start_time

    X_train_new = pd.concat([X_train_diveplane_2, gen_df], ignore_index=True, axis=0)
    y_train_new = X_train_new.iloc[:, -1]
    X_train_new = X_train_new.iloc[:, :-1]
    
    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)
    
    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train_new)
    clf_KNN.fit(X_train_scale, y_train_new)
    clf_LR.fit(X_train_scale, y_train_new)
    clf_DT.fit(X_train_scale, y_train_new)
    clf_RF.fit(X_train_scale, y_train_new)
    clf_LightGBM.fit(X_train_scale, y_train_new)
    clf_Adaboost.fit(X_train_scale, y_train_new)
    clf_GBDT.fit(X_train_scale, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["Diveplane2", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["Diveplane2", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["Diveplane2", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["Diveplane2", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["Diveplane2", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["Diveplane2", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["Diveplane2", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["Diveplane2", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
    
    ########## DS run ##########
    print("----- DS -----")
    mode = "independent_attribute_mode"

    col = X_train.columns
    tar = y_train.name
    
    X_train_DS = X_train.copy()
    y_train_DS = y_train.copy()
    X_train_DS[tar] = y_train_DS
    write_df = X_train_DS[X_train_DS[tar] == 1]
    write_df = write_df.iloc[:, :-1]
    write_df.to_csv(f"{os.path.dirname(os.getcwd())}/extra/js_vuln_pos_df.csv", index=False)
    
    threshold = 20
    num_tuples_to_generate = int(y_train.value_counts()[0] - y_train.value_counts()[1])

    start_time = time.time()

    description_file = f"{os.path.dirname(os.getcwd())}/extra/js_vuln.json"
    describer = DataDescriber(category_threshold=threshold)
    describer.describe_dataset_in_independent_attribute_mode(
        dataset_file=f"{os.path.dirname(os.getcwd())}/extra/js_vuln_pos_df.csv"
    )
    describer.save_dataset_description_to_file(description_file)

    generator = DataGenerator()
    generator.generate_dataset_in_independent_mode(num_tuples_to_generate, description_file)
    generator.save_synthetic_data(f"{os.path.dirname(os.getcwd())}/extra/js_vuln_syn_df.csv")

    rt = time.time() - start_time

    X_train_new = pd.read_csv(f"{os.path.dirname(os.getcwd())}/extra/js_vuln_syn_df.csv").to_numpy()
    y_train_new = np.ones(num_tuples_to_generate)
    X_train_new = pd.DataFrame(np.vstack((X_train.to_numpy(), X_train_new)), columns=col)
    y_train_new = pd.Series(np.hstack((y_train.to_numpy(), y_train_new)), name=tar)
    
    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train_new)
    clf_KNN.fit(X_train_scale, y_train_new)
    clf_LR.fit(X_train_scale, y_train_new)
    clf_DT.fit(X_train_scale, y_train_new)
    clf_RF.fit(X_train_scale, y_train_new)
    clf_LightGBM.fit(X_train_scale, y_train_new)
    clf_Adaboost.fit(X_train_scale, y_train_new)
    clf_GBDT.fit(X_train_scale, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["DS", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["DS", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["DS", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["DS", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["DS", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["DS", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["DS", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["DS", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
    
    ########## SDV FASTML run ##########
    print("----- SDV FASTML -----")
    col = X_train.columns
    tar = y_train.name
    num_tuples_to_generate = int(y_train.value_counts()[0] - y_train.value_counts()[1])
    
    X_train_SDV = X_train.copy()
    y_train_SDV = y_train.copy()
    X_train_SDV[tar] = y_train_SDV
    pos_df = X_train_SDV[X_train_SDV[tar] == 1]
    pos_df = pos_df.iloc[:, :-1]

    metadata = SingleTableMetadata()
    metadata.detect_from_dataframe(data=pos_df)
    
    start_time = time.time()
    syn1 = SingleTablePreset(metadata, name="FAST_ML")
    syn1.fit(data=pos_df)
    X_train_new = syn1.sample(num_rows=num_tuples_to_generate).to_numpy()

    rt = time.time() - start_time

    X_train_new = pd.DataFrame(np.vstack((X_train.to_numpy(), X_train_new)), columns=col)
    y_train_new = np.ones(num_tuples_to_generate)
    y_train_new = pd.Series(np.hstack((y_train.to_numpy(), y_train_new)), name=tar)

    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train_new)
    clf_KNN.fit(X_train_scale, y_train_new)
    clf_LR.fit(X_train_scale, y_train_new)
    clf_DT.fit(X_train_scale, y_train_new)
    clf_RF.fit(X_train_scale, y_train_new)
    clf_LightGBM.fit(X_train_scale, y_train_new)
    clf_Adaboost.fit(X_train_scale, y_train_new)
    clf_GBDT.fit(X_train_scale, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["SDV_FASTML", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["SDV_FASTML", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["SDV_FASTML", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["SDV_FASTML", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["SDV_FASTML", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["SDV_FASTML", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["SDV_FASTML", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["SDV_FASTML", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
    
    ########## SDV GC ##########
    print("----- SDV GC -----")
    start_time = time.time()
    syn2 = GaussianCopulaSynthesizer(metadata)
    syn2.fit(data=pos_df)
    X_train_new = syn2.sample(num_rows=num_tuples_to_generate).to_numpy()

    rt = time.time() - start_time

    X_train_new = pd.DataFrame(np.vstack((X_train.to_numpy(), X_train_new)), columns=col)
    y_train_new = np.ones(num_tuples_to_generate)
    y_train_new = pd.Series(np.hstack((y_train.to_numpy(), y_train_new)), name=tar)

    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train_new)
    clf_KNN.fit(X_train_scale, y_train_new)
    clf_LR.fit(X_train_scale, y_train_new)
    clf_DT.fit(X_train_scale, y_train_new)
    clf_RF.fit(X_train_scale, y_train_new)
    clf_LightGBM.fit(X_train_scale, y_train_new)
    clf_Adaboost.fit(X_train_scale, y_train_new)
    clf_GBDT.fit(X_train_scale, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["SDV_GC", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["SDV_GC", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["SDV_GC", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["SDV_GC", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["SDV_GC", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["SDV_GC", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["SDV_GC", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["SDV_GC", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))
    
    ########## SDV GAN ##########
    print("----- SDV GAN -----")
    start_time = time.time()
    syn3 = CTGANSynthesizer(metadata)
    syn3.fit(data=pos_df)
    X_train_new = syn3.sample(num_rows=num_tuples_to_generate).to_numpy()

    rt = time.time() - start_time

    X_train_new = pd.DataFrame(np.vstack((X_train.to_numpy(), X_train_new)), columns=col)
    y_train_new = np.ones(num_tuples_to_generate)
    y_train_new = pd.Series(np.hstack((y_train.to_numpy(), y_train_new)), name=tar)

    # scale data
    scaler = StandardScaler()
    X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_scale, y_train_new)
    clf_KNN.fit(X_train_scale, y_train_new)
    clf_LR.fit(X_train_scale, y_train_new)
    clf_DT.fit(X_train_scale, y_train_new)
    clf_RF.fit(X_train_scale, y_train_new)
    clf_LightGBM.fit(X_train_scale, y_train_new)
    clf_Adaboost.fit(X_train_scale, y_train_new)
    clf_GBDT.fit(X_train_scale, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)
    
    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["SDV_GAN", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["SDV_GAN", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["SDV_GAN", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["SDV_GAN", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["SDV_GAN", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["SDV_GAN", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["SDV_GAN", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["SDV_GAN", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))

----- in repeat 4 -----
y value counts: 
 0    1288
1     209
Name: isBug, dtype: int64
y class ratio: 1: 6
--- y train classes count: 
0    1030
1     167
Name: isBug, dtype: int64
--- y train ratio: 1:6
 
--- y test classes count: 
0    258
1     42
Name: isBug, dtype: int64
--- y test ratio: 1:6
----- normal -----
----- random -----
y train ratio: 1:1
----- ADASYN ------
y train ratio: 1:1
----- borderlineSMOTE -----
y train ratio: 1:1
----- SMOTE -----
y train ratio: 1:1
----- SVMSMOTE -----
y train ratio: 1:1
----- SMOTUNED -----
y train ratio of SVM: 1:1
y train ratio of KNN: 1:1
y train ratio of LR: 1:1
y train ratio of DT: 1:1
y train ratio of RF: 1:1
y train ratio of LightGBM: 1:1
y train ratio of Adaboost: 1:1
y train ratio of GBDT: 1:1
----- DAZZLE1 -----
[    0/29700] LG:0.408 LD:0.076 D:0.147 GP:0.006 RMSEAVG:0.329 NUM:0.329 SynTraiAuc:0.429 RFAcc:1.000                                                                                                            
[  500/29700]

863                                                                                                                                                                                                              
[    0/8436] LG:0.159 LD:-0.178 D:0.226 GP:0.025 RMSEAVG:0.319 NUM:0.319 SynTraiAuc:0.416 RFAcc:1.000                                                                                                            
[  500/8436] LG:1.027 LD:2.266 D:2.363 GP:0.006 RMSEAVG:0.319 NUM:0.319 SynTraiAuc:0.416 RFAcc:1.000                                                                                                             
[ 1000/8436] LG:1.114 LD:1.871 D:1.893 GP:0.001 RMSEAVG:0.319 NUM:0.319 SynTraiAuc:0.416 RFAcc:1.000                                                                                                             
[ 1500/8436] LG:1.166 LD:1.501 D:1.527 GP:0.002 RMSEAVG:0.141 NUM:0.141 SynTraiAuc:0.416 RFAcc:1.000                                                            

[19500/24704] LG:1.961 LD:0.372 D:0.433 GP:0.004 RMSEAVG:0.035 NUM:0.035 SynTraiAuc:0.803 RFAcc:0.974                                                                                                            
[20000/24704] LG:1.886 LD:0.364 D:0.447 GP:0.006 RMSEAVG:0.021 NUM:0.021 SynTraiAuc:0.624 RFAcc:0.976                                                                                                            
[20500/24704] LG:1.931 LD:0.593 D:0.632 GP:0.003 RMSEAVG:0.021 NUM:0.021 SynTraiAuc:0.624 RFAcc:0.976                                                                                                            
[21000/24704] LG:1.977 LD:0.495 D:0.542 GP:0.003 RMSEAVG:0.021 NUM:0.021 SynTraiAuc:0.624 RFAcc:0.976                                                                                                            
[21500/24704] LG:1.931 LD:0.415 D:0.493 GP:0.006 RMSEAVG:0.031 NUM:0.031 SynTraiAuc:0.624 RFAcc:0.982                                                           

[11000/11072] LG:1.354 LD:0.975 D:1.071 GP:0.011 RMSEAVG:0.037 NUM:0.037 SynTraiAuc:0.683 RFAcc:0.990                                                                                                            
[11072/11072] LG:1.577 LD:0.745 D:1.001 GP:0.028 RMSEAVG:0.037 NUM:0.037 SynTraiAuc:0.683 RFAcc:0.990                                                                                                            
needed sample                                                                                                                                                                                                    
723                                                                                                                                                                                                              
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [04:58<00

2023-10-09 19:05:54.149947: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-09 19:05:54.150172: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


...

y train ratio: 1:1
----- SDV GAN -----


/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/numerical.py:112: UserWarning: No rounding scheme detected for column 'BonPwr'. Data will not be rounded.
  warnings.warn(
/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/numerical.py:112: UserWarning: No rounding scheme detected for column 'Between'. Data will not be rounded.
  warnings.warn(
/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/home/lingxiao/diveplane/lib/python3.8/site-packa

/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missi

/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missi

y train ratio: 1:1


In [2]:
rnl = [437, 66, 227, 89, 126, 238, 220, 135, 248, 103]

for idx, rn in enumerate(rnl):
#     write_file = f"Defect_Eclipse_JDT_Core_res_r{idx+1}_rn{rn}.csv"
#     write_path = f"{os.path.dirname(os.getcwd())}/result/Defect_Eclipse_JDT_Core/{write_file}"
    write_file = f"Defect_Mylyn_res_r{idx+1}_rn{rn}.csv"
    write_path = f"{os.path.dirname(os.getcwd())}/result/Defect_Mylyn/{write_file}"
    
    data_path = f"{os.path.dirname(os.getcwd())}/data/imbalance_defects_prediction/7_CK_NET_PROC/input/Mylyn--CK_NET_PROC.arff"
    data = arff.loadarff(data_path)
    df = pd.DataFrame(data[0])
    df['isBug'] = df['isBug'].astype('str')
    d = {'b\'YES\'': 1, 'b\'NO\'': 0}
    df['isBug'] = df['isBug'].astype(str).map(d).fillna(df['isBug'])
    df = df.drop_duplicates()
    df.reset_index(inplace=True, drop=True)

    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    print("y value counts: \n", str(y.value_counts()))
    print("y class ratio: 1:", str(round(y.value_counts()[0]/y.value_counts()[1])))

    cols = X.columns
    tar = y.name

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=rn)
    print("--- y train classes count: \n" + str(y_train.value_counts()))
    print("--- y train ratio: 1:" + str(round(y_train.value_counts()[0] / y_train.value_counts()[1])))
    print(" ")
    print("--- y test classes count: \n" + str(y_test.value_counts()))
    print("--- y test ratio: 1:" + str(round(y_test.value_counts()[0] / y_test.value_counts()[1])))

    ########## DAZZLE run ##########
    # DAZZLE oversampling run - DAZZLE oversampling technique
    print("----- DAZZLE1 -----")

    X_train_GAN = X_train.copy()
    y_train_GAN = y_train.copy()
    X_test_GAN = X_test.copy()
    y_test_GAN = y_test.copy()
    rt, X_train_new, y_train_new, X_test_scale = DAZZLE1Oversampling(X_train=X_train_GAN, 
                                                                     y_train=y_train_GAN, 
                                                                     X_test=X_test_GAN,
                                                                     y_test=y_test_GAN)

    X_train_new = pd.DataFrame(X_train_new, columns=cols)
    y_train_new = pd.Series(y_train_new, name=tar)
    X_test_scale = pd.DataFrame(X_test_scale, columns=cols)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_new, y_train_new)
    clf_KNN.fit(X_train_new, y_train_new)
    clf_LR.fit(X_train_new, y_train_new)
    clf_DT.fit(X_train_new, y_train_new)
    clf_RF.fit(X_train_new, y_train_new)
    clf_LightGBM.fit(X_train_new, y_train_new)
    clf_Adaboost.fit(X_train_new, y_train_new)
    clf_GBDT.fit(X_train_new, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)

    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["DAZZLE1", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["DAZZLE1", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["DAZZLE1", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["DAZZLE1", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["DAZZLE1", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["DAZZLE1", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["DAZZLE1", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["DAZZLE1", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))

y value counts: 
 0    1616
1     245
Name: isBug, dtype: int64
y class ratio: 1: 7
--- y train classes count: 
0    1292
1     196
Name: isBug, dtype: int64
--- y train ratio: 1:7
 
--- y test classes count: 
0    324
1     49
Name: isBug, dtype: int64
--- y test ratio: 1:7
----- DAZZLE1 -----
[    0/15066] LG:0.261 LD:-1.147 D:-0.112 GP:0.207 RMSEAVG:0.273 NUM:0.273 SynTraiAuc:0.508 RFAcc:1.000                 
[  500/15066] LG:0.347 LD:0.838 D:0.981 GP:0.028 RMSEAVG:0.273 NUM:0.273 SynTraiAuc:0.508 RFAcc:1.000                   
[ 1000/15066] LG:0.606 LD:1.657 D:2.003 GP:0.069 RMSEAVG:0.273 NUM:0.273 SynTraiAuc:0.508 RFAcc:1.000                   
[ 1500/15066] LG:0.434 LD:1.107 D:1.183 GP:0.015 RMSEAVG:0.131 NUM:0.131 SynTraiAuc:0.508 RFAcc:1.000                   
[ 2000/15066] LG:0.562 LD:0.958 D:1.200 GP:0.048 RMSEAVG:0.131 NUM:0.131 SynTraiAuc:0.508 RFAcc:1.000                   
[ 2500/15066] LG:0.691 LD:1.049 D:1.161 GP:0.022 RMSEAVG:0.052 NUM:0.052 SynTraiAuc:0.508 RFAcc:0.9

[  500/3300] LG:1.918 LD:2.013 D:2.158 GP:0.024 RMSEAVG:0.260 NUM:0.260 SynTraiAuc:0.447 RFAcc:1.000                    
[ 1000/3300] LG:2.084 LD:1.925 D:2.056 GP:0.022 RMSEAVG:0.260 NUM:0.260 SynTraiAuc:0.447 RFAcc:1.000                    
[ 1500/3300] LG:1.986 LD:1.596 D:1.689 GP:0.016 RMSEAVG:0.115 NUM:0.115 SynTraiAuc:0.447 RFAcc:1.000                    
[ 2000/3300] LG:1.876 LD:1.333 D:1.385 GP:0.009 RMSEAVG:0.115 NUM:0.115 SynTraiAuc:0.447 RFAcc:1.000                    
[ 2500/3300] LG:2.022 LD:1.187 D:1.307 GP:0.020 RMSEAVG:0.028 NUM:0.028 SynTraiAuc:0.447 RFAcc:0.979                    
[ 3000/3300] LG:1.917 LD:1.157 D:1.281 GP:0.021 RMSEAVG:0.028 NUM:0.028 SynTraiAuc:0.447 RFAcc:0.979                    
[ 3300/3300] LG:1.949 LD:1.168 D:1.239 GP:0.012 RMSEAVG:0.028 NUM:0.028 SynTraiAuc:0.447 RFAcc:0.979                    
needed sample                                                                                                           
1096                            

[ 7000/24366] LG:1.024 LD:0.428 D:0.497 GP:0.009 RMSEAVG:0.029 NUM:0.029 SynTraiAuc:0.569 RFAcc:0.956                   
[ 7500/24366] LG:0.923 LD:0.795 D:0.896 GP:0.013 RMSEAVG:0.030 NUM:0.030 SynTraiAuc:0.569 RFAcc:0.961                   
[ 8000/24366] LG:0.863 LD:0.580 D:0.593 GP:0.002 RMSEAVG:0.030 NUM:0.030 SynTraiAuc:0.569 RFAcc:0.961                   
[ 8500/24366] LG:0.764 LD:0.513 D:0.550 GP:0.005 RMSEAVG:0.030 NUM:0.030 SynTraiAuc:0.569 RFAcc:0.961                   
[ 9000/24366] LG:0.888 LD:0.638 D:0.730 GP:0.011 RMSEAVG:0.025 NUM:0.025 SynTraiAuc:0.569 RFAcc:0.968                   
[ 9500/24366] LG:0.857 LD:0.611 D:0.659 GP:0.006 RMSEAVG:0.025 NUM:0.025 SynTraiAuc:0.569 RFAcc:0.968                   
[10000/24366] LG:0.890 LD:0.588 D:0.613 GP:0.003 RMSEAVG:0.026 NUM:0.026 SynTraiAuc:0.704 RFAcc:0.963                   
[10500/24366] LG:1.010 LD:0.644 D:0.696 GP:0.007 RMSEAVG:0.026 NUM:0.026 SynTraiAuc:0.704 RFAcc:0.963                   
[11000/24366] LG:1.086 LD:0.628 

[ 1500/5781] LG:0.783 LD:1.105 D:1.175 GP:0.005 RMSEAVG:0.076 NUM:0.076 SynTraiAuc:0.279 RFAcc:0.994                    
[ 2000/5781] LG:0.849 LD:1.032 D:1.125 GP:0.007 RMSEAVG:0.076 NUM:0.076 SynTraiAuc:0.279 RFAcc:0.994                    
[ 2500/5781] LG:0.898 LD:0.992 D:1.036 GP:0.003 RMSEAVG:0.047 NUM:0.047 SynTraiAuc:0.279 RFAcc:0.991                    
[ 3000/5781] LG:0.985 LD:1.050 D:1.137 GP:0.006 RMSEAVG:0.047 NUM:0.047 SynTraiAuc:0.279 RFAcc:0.991                    
[ 3500/5781] LG:1.060 LD:0.972 D:1.038 GP:0.005 RMSEAVG:0.047 NUM:0.047 SynTraiAuc:0.279 RFAcc:0.991                    
[ 4000/5781] LG:0.933 LD:0.939 D:1.018 GP:0.006 RMSEAVG:0.037 NUM:0.037 SynTraiAuc:0.279 RFAcc:0.990                    
[ 4500/5781] LG:0.971 LD:0.842 D:0.892 GP:0.004 RMSEAVG:0.037 NUM:0.037 SynTraiAuc:0.279 RFAcc:0.990                    
[ 5000/5781] LG:0.880 LD:0.847 D:0.983 GP:0.010 RMSEAVG:0.034 NUM:0.034 SynTraiAuc:0.755 RFAcc:0.979                    
[ 5500/5781] LG:0.833 LD:0.777 D

[ 3000/32364] LG:0.514 LD:0.831 D:0.885 GP:0.008 RMSEAVG:0.038 NUM:0.038 SynTraiAuc:0.482 RFAcc:0.990                   
[ 3500/32364] LG:0.647 LD:0.881 D:0.926 GP:0.006 RMSEAVG:0.038 NUM:0.038 SynTraiAuc:0.482 RFAcc:0.990                   
[ 4000/32364] LG:0.704 LD:0.836 D:0.906 GP:0.010 RMSEAVG:0.032 NUM:0.032 SynTraiAuc:0.482 RFAcc:0.967                   
[ 4500/32364] LG:0.669 LD:0.749 D:0.831 GP:0.012 RMSEAVG:0.032 NUM:0.032 SynTraiAuc:0.482 RFAcc:0.967                   
[ 5000/32364] LG:0.614 LD:0.682 D:0.698 GP:0.002 RMSEAVG:0.031 NUM:0.031 SynTraiAuc:0.596 RFAcc:0.960                   
[ 5500/32364] LG:0.790 LD:0.899 D:1.060 GP:0.023 RMSEAVG:0.031 NUM:0.031 SynTraiAuc:0.596 RFAcc:0.960                   
[ 6000/32364] LG:0.708 LD:0.907 D:0.989 GP:0.012 RMSEAVG:0.031 NUM:0.031 SynTraiAuc:0.596 RFAcc:0.960                   
[ 6500/32364] LG:0.735 LD:0.470 D:0.514 GP:0.006 RMSEAVG:0.048 NUM:0.048 SynTraiAuc:0.596 RFAcc:0.971                   
[ 7000/32364] LG:0.800 LD:0.662 

...

[    0/21018] LG:-0.094 LD:-1.458 D:-0.220 GP:0.095 RMSEAVG:0.297 NUM:0.297 SynTraiAuc:0.604 RFAcc:1.000                
[  500/21018] LG:1.026 LD:1.721 D:1.813 GP:0.007 RMSEAVG:0.297 NUM:0.297 SynTraiAuc:0.604 RFAcc:1.000                   
[ 1000/21018] LG:1.220 LD:1.655 D:1.764 GP:0.008 RMSEAVG:0.297 NUM:0.297 SynTraiAuc:0.604 RFAcc:1.000                   
[ 1500/21018] LG:1.167 LD:1.738 D:1.826 GP:0.007 RMSEAVG:0.187 NUM:0.187 SynTraiAuc:0.604 RFAcc:1.000                   
[ 2000/21018] LG:1.080 LD:1.754 D:1.880 GP:0.010 RMSEAVG:0.187 NUM:0.187 SynTraiAuc:0.604 RFAcc:1.000                   
[ 2500/21018] LG:0.950 LD:1.551 D:1.640 GP:0.007 RMSEAVG:0.110 NUM:0.110 SynTraiAuc:0.604 RFAcc:0.999                   
[ 3000/21018] LG:0.827 LD:1.256 D:1.278 GP:0.002 RMSEAVG:0.110 NUM:0.110 SynTraiAuc:0.604 RFAcc:0.999                   
[ 3500/21018] LG:0.754 LD:1.200 D:1.271 GP:0.005 RMSEAVG:0.110 NUM:0.110 SynTraiAuc:0.604 RFAcc:0.999                   
[ 4000/21018] LG:0.854 LD:1.100 

[13000/20274] LG:1.186 LD:0.522 D:0.580 GP:0.003 RMSEAVG:0.027 NUM:0.027 SynTraiAuc:0.672 RFAcc:0.936                   
[13500/20274] LG:0.944 LD:0.558 D:0.622 GP:0.004 RMSEAVG:0.027 NUM:0.027 SynTraiAuc:0.672 RFAcc:0.936                   
[14000/20274] LG:1.063 LD:0.591 D:0.654 GP:0.003 RMSEAVG:0.033 NUM:0.033 SynTraiAuc:0.672 RFAcc:0.979                   
[14500/20274] LG:1.167 LD:0.590 D:0.677 GP:0.005 RMSEAVG:0.033 NUM:0.033 SynTraiAuc:0.672 RFAcc:0.979                   
[15000/20274] LG:1.165 LD:0.628 D:0.708 GP:0.004 RMSEAVG:0.033 NUM:0.033 SynTraiAuc:0.516 RFAcc:0.951                   
[15500/20274] LG:1.150 LD:0.612 D:0.717 GP:0.006 RMSEAVG:0.033 NUM:0.033 SynTraiAuc:0.516 RFAcc:0.951                   
[16000/20274] LG:1.165 LD:0.450 D:0.518 GP:0.004 RMSEAVG:0.033 NUM:0.033 SynTraiAuc:0.516 RFAcc:0.951                   
[16500/20274] LG:1.054 LD:0.713 D:0.749 GP:0.002 RMSEAVG:0.036 NUM:0.036 SynTraiAuc:0.516 RFAcc:0.946                   
[17000/20274] LG:1.280 LD:0.559 

[10000/24087] LG:0.988 LD:0.555 D:0.664 GP:0.007 RMSEAVG:0.034 NUM:0.034 SynTraiAuc:0.609 RFAcc:0.983                   
[10500/24087] LG:0.661 LD:0.435 D:0.518 GP:0.005 RMSEAVG:0.034 NUM:0.034 SynTraiAuc:0.609 RFAcc:0.983                   
[11000/24087] LG:1.246 LD:0.987 D:1.119 GP:0.008 RMSEAVG:0.034 NUM:0.034 SynTraiAuc:0.609 RFAcc:0.983                   
[11500/24087] LG:0.462 LD:0.564 D:0.663 GP:0.006 RMSEAVG:0.085 NUM:0.085 SynTraiAuc:0.609 RFAcc:0.993                   
[12000/24087] LG:0.743 LD:0.913 D:0.994 GP:0.005 RMSEAVG:0.085 NUM:0.085 SynTraiAuc:0.609 RFAcc:0.993                   
[12500/24087] LG:0.931 LD:1.034 D:1.106 GP:0.004 RMSEAVG:0.071 NUM:0.071 SynTraiAuc:0.609 RFAcc:0.977                   
[13000/24087] LG:0.611 LD:1.003 D:1.138 GP:0.008 RMSEAVG:0.071 NUM:0.071 SynTraiAuc:0.609 RFAcc:0.977                   
[13500/24087] LG:0.658 LD:1.352 D:1.444 GP:0.006 RMSEAVG:0.071 NUM:0.071 SynTraiAuc:0.609 RFAcc:0.977                   
[14000/24087] LG:0.378 LD:0.851 

needed sample                                                                                                           
1096                                                                                                                    
[    0/15839] LG:0.340 LD:-0.075 D:0.148 GP:0.037 RMSEAVG:0.246 NUM:0.246 SynTraiAuc:0.386 RFAcc:1.000                  
[  500/15839] LG:1.668 LD:1.944 D:2.033 GP:0.015 RMSEAVG:0.246 NUM:0.246 SynTraiAuc:0.386 RFAcc:1.000                   
[ 1000/15839] LG:1.688 LD:1.513 D:1.635 GP:0.020 RMSEAVG:0.246 NUM:0.246 SynTraiAuc:0.386 RFAcc:1.000                   
[ 1500/15839] LG:1.627 LD:1.131 D:1.367 GP:0.039 RMSEAVG:0.049 NUM:0.049 SynTraiAuc:0.386 RFAcc:0.987                   
[ 2000/15839] LG:1.608 LD:1.075 D:1.193 GP:0.020 RMSEAVG:0.049 NUM:0.049 SynTraiAuc:0.386 RFAcc:0.987                   
[ 2500/15839] LG:1.528 LD:1.206 D:1.287 GP:0.013 RMSEAVG:0.047 NUM:0.047 SynTraiAuc:0.386 RFAcc:0.986                   
[ 3000/15839] LG:1.583 LD:1.038 

[12500/17061] LG:1.563 LD:0.797 D:0.872 GP:0.015 RMSEAVG:0.023 NUM:0.023 SynTraiAuc:0.747 RFAcc:0.981                   
[13000/17061] LG:1.562 LD:0.720 D:0.828 GP:0.022 RMSEAVG:0.023 NUM:0.023 SynTraiAuc:0.747 RFAcc:0.981                   
[13500/17061] LG:1.645 LD:0.761 D:0.803 GP:0.008 RMSEAVG:0.023 NUM:0.023 SynTraiAuc:0.747 RFAcc:0.981                   
[14000/17061] LG:1.590 LD:0.754 D:0.784 GP:0.006 RMSEAVG:0.021 NUM:0.021 SynTraiAuc:0.747 RFAcc:0.938                   
[14500/17061] LG:1.609 LD:0.684 D:0.737 GP:0.010 RMSEAVG:0.021 NUM:0.021 SynTraiAuc:0.747 RFAcc:0.938                   
[15000/17061] LG:1.689 LD:0.663 D:0.700 GP:0.007 RMSEAVG:0.026 NUM:0.026 SynTraiAuc:0.729 RFAcc:0.966                   
[15500/17061] LG:1.659 LD:0.844 D:0.906 GP:0.012 RMSEAVG:0.026 NUM:0.026 SynTraiAuc:0.729 RFAcc:0.966                   
[16000/17061] LG:1.650 LD:0.750 D:0.812 GP:0.012 RMSEAVG:0.026 NUM:0.026 SynTraiAuc:0.729 RFAcc:0.966                   
[16500/17061] LG:1.621 LD:0.865 

In [3]:
rnl = [437, 66, 227, 89, 126, 238, 220, 135, 248, 103]

for idx, rn in enumerate(rnl):
#     write_file = f"Defect_Eclipse_JDT_Core_res_r{idx+1}_rn{rn}.csv"
#     write_path = f"{os.path.dirname(os.getcwd())}/result/Defect_Eclipse_JDT_Core/{write_file}"
    
    write_file = f"Defect_Mylyn_res_r{idx+1}_rn{rn}.csv"
    write_path = f"{os.path.dirname(os.getcwd())}/result/Defect_Mylyn/{write_file}"
    
    data_path = f"{os.path.dirname(os.getcwd())}/data/imbalance_defects_prediction/7_CK_NET_PROC/input/Mylyn--CK_NET_PROC.arff"
    data = arff.loadarff(data_path)
    df = pd.DataFrame(data[0])
    df['isBug'] = df['isBug'].astype('str')
    d = {'b\'YES\'': 1, 'b\'NO\'': 0}
    df['isBug'] = df['isBug'].astype(str).map(d).fillna(df['isBug'])
    df = df.drop_duplicates()
    df.reset_index(inplace=True, drop=True)
    
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    print("y value counts: \n", str(y.value_counts()))
    print("y class ratio: 1:", str(round(y.value_counts()[0]/y.value_counts()[1])))

    cols = X.columns
    tar = y.name

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=rn)
    print("--- y train classes count: \n" + str(y_train.value_counts()))
    print("--- y train ratio: 1:" + str(round(y_train.value_counts()[0] / y_train.value_counts()[1])))
    print(" ")
    print("--- y test classes count: \n" + str(y_test.value_counts()))
    print("--- y test ratio: 1:" + str(round(y_test.value_counts()[0] / y_test.value_counts()[1])))

    ########## DAZZLE run ##########
    # DAZZLE oversampling run - DAZZLE oversampling technique
    print("----- DAZZLE2 -----")

    X_train_GAN = X_train.copy()
    y_train_GAN = y_train.copy()
    X_test_GAN = X_test.copy()
    y_test_GAN = y_test.copy()
    rt, X_train_new, y_train_new, X_test_scale = DAZZLE2Oversampling(X_train=X_train_GAN, 
                                                                     y_train=y_train_GAN, 
                                                                     X_test=X_test_GAN)

    X_train_new = pd.DataFrame(X_train_new, columns=cols)
    y_train_new = pd.Series(y_train_new, name=tar)
    X_test_scale = pd.DataFrame(X_test_scale, columns=cols)
    # scale data
    # scaler = StandardScaler()
    # X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
    # X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

    # create models
    clf_SVM = SVC()
    clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
    clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
    clf_DT = DecisionTreeClassifier()
    clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
    clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
    clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
    clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

    clf_SVM.fit(X_train_new, y_train_new)
    clf_KNN.fit(X_train_new, y_train_new)
    clf_LR.fit(X_train_new, y_train_new)
    clf_DT.fit(X_train_new, y_train_new)
    clf_RF.fit(X_train_new, y_train_new)
    clf_LightGBM.fit(X_train_new, y_train_new)
    clf_Adaboost.fit(X_train_new, y_train_new)
    clf_GBDT.fit(X_train_new, y_train_new)

    y_pred_SVM = clf_SVM.predict(X_test_scale)
    y_pred_KNN = clf_KNN.predict(X_test_scale)
    y_pred_LR = clf_LR.predict(X_test_scale)
    y_pred_DT = clf_DT.predict(X_test_scale)
    y_pred_RF = clf_RF.predict(X_test_scale)
    y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
    y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
    y_pred_GBDT = clf_GBDT.predict(X_test_scale)

    with open(write_path, "a", newline="") as f:
        csv_writer = csv.writer(f)

        csv_writer.writerow(["DAZZLE2", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
        csv_writer.writerow(["DAZZLE2", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
        csv_writer.writerow(["DAZZLE2", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
        csv_writer.writerow(["DAZZLE2", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
        csv_writer.writerow(["DAZZLE2", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
        csv_writer.writerow(["DAZZLE2", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
        csv_writer.writerow(["DAZZLE2", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
        csv_writer.writerow(["DAZZLE2", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))

y value counts: 
 0    1616
1     245
Name: isBug, dtype: int64
y class ratio: 1: 7
--- y train classes count: 
0    1292
1     196
Name: isBug, dtype: int64
--- y train ratio: 1:7
 
--- y test classes count: 
0    324
1     49
Name: isBug, dtype: int64
--- y test ratio: 1:7
----- DAZZLE2 -----
[    0/26149] LG:0.115 LD:-0.015 D:0.267 GP:0.047 RMSEAVG:0.308 NUM:0.308 SynTraiAuc:0.417 RFAcc:1.000                  
[  500/26149] LG:1.280 LD:1.646 D:1.743 GP:0.016 RMSEAVG:0.308 NUM:0.308 SynTraiAuc:0.417 RFAcc:1.000                   
[ 1000/26149] LG:1.345 LD:1.208 D:1.358 GP:0.025 RMSEAVG:0.308 NUM:0.308 SynTraiAuc:0.417 RFAcc:1.000                   
[ 1500/26149] LG:1.057 LD:1.190 D:1.331 GP:0.023 RMSEAVG:0.059 NUM:0.059 SynTraiAuc:0.417 RFAcc:0.978                   
[ 2000/26149] LG:1.044 LD:0.960 D:1.035 GP:0.013 RMSEAVG:0.059 NUM:0.059 SynTraiAuc:0.417 RFAcc:0.978                   
[ 2500/26149] LG:0.940 LD:0.968 D:1.104 GP:0.023 RMSEAVG:0.033 NUM:0.033 SynTraiAuc:0.417 RFAcc:0.9

[ 8500/8927] LG:0.782 LD:-0.691 D:0.032 GP:0.038 RMSEAVG:0.148 NUM:0.148 SynTraiAuc:0.656 RFAcc:0.998                   
[ 8927/8927] LG:1.079 LD:0.374 D:0.496 GP:0.006 RMSEAVG:0.221 NUM:0.221 SynTraiAuc:0.656 RFAcc:1.000                    
needed sample                                                                                                           
930                                                                                                                     
100%|██████████████████████████████████████████████████| 5/5 [10:28<00:00, 125.71s/trial, best loss: 0.6574455562095349]
[ 5000/10509] LG:0.206 LD:-0.185 D:0.024 GP:0.011 RMSEAVG:0.141 NUM:0.141 SynTraiAuc:0.634 RFAcc:1.000    
[10000/10509] LG:-0.012 LD:-0.229 D:-0.098 GP:0.007 RMSEAVG:0.099 NUM:0.099 SynTraiAuc:0.699 RFAcc:1.000  
[10509/10509] LG:-0.359 LD:0.124 D:0.128 GP:0.000 RMSEAVG:0.099 NUM:0.099 SynTraiAuc:0.699 RFAcc:1.000  
needed sample 1096
y train ratio: 1:1
y value counts: 
 0    1616
1     245


[ 3000/15680] LG:1.063 LD:0.930 D:1.015 GP:0.007 RMSEAVG:0.033 NUM:0.033 SynTraiAuc:0.444 RFAcc:0.981                   
[ 3500/15680] LG:1.120 LD:0.905 D:0.988 GP:0.006 RMSEAVG:0.033 NUM:0.033 SynTraiAuc:0.444 RFAcc:0.981                   
[ 4000/15680] LG:1.176 LD:0.885 D:0.939 GP:0.004 RMSEAVG:0.030 NUM:0.030 SynTraiAuc:0.444 RFAcc:0.945                   
[ 4500/15680] LG:1.220 LD:0.950 D:1.066 GP:0.009 RMSEAVG:0.030 NUM:0.030 SynTraiAuc:0.444 RFAcc:0.945                   
[ 5000/15680] LG:1.206 LD:0.791 D:0.905 GP:0.009 RMSEAVG:0.027 NUM:0.027 SynTraiAuc:0.666 RFAcc:0.964                   
[ 5500/15680] LG:1.208 LD:0.783 D:0.841 GP:0.004 RMSEAVG:0.027 NUM:0.027 SynTraiAuc:0.666 RFAcc:0.964                   
[ 6000/15680] LG:1.349 LD:0.789 D:0.829 GP:0.003 RMSEAVG:0.027 NUM:0.027 SynTraiAuc:0.666 RFAcc:0.964                   
[ 6500/15680] LG:1.187 LD:0.673 D:0.723 GP:0.004 RMSEAVG:0.027 NUM:0.027 SynTraiAuc:0.666 RFAcc:0.972                   
[ 7000/15680] LG:1.155 LD:0.687 

[ 9500/24727] LG:1.042 LD:0.268 D:0.347 GP:0.005 RMSEAVG:0.103 NUM:0.103 SynTraiAuc:0.512 RFAcc:0.970                   
[10000/24727] LG:1.161 LD:0.318 D:0.442 GP:0.008 RMSEAVG:0.067 NUM:0.067 SynTraiAuc:0.481 RFAcc:0.978                   
[10500/24727] LG:1.077 LD:0.375 D:0.426 GP:0.003 RMSEAVG:0.067 NUM:0.067 SynTraiAuc:0.481 RFAcc:0.978                   
[11000/24727] LG:1.242 LD:0.343 D:0.349 GP:0.000 RMSEAVG:0.067 NUM:0.067 SynTraiAuc:0.481 RFAcc:0.978                   
[11500/24727] LG:1.255 LD:0.264 D:0.310 GP:0.003 RMSEAVG:0.051 NUM:0.051 SynTraiAuc:0.481 RFAcc:0.973                   
[12000/24727] LG:1.145 LD:0.402 D:0.425 GP:0.001 RMSEAVG:0.051 NUM:0.051 SynTraiAuc:0.481 RFAcc:0.973                   
[12500/24727] LG:1.144 LD:0.190 D:0.302 GP:0.007 RMSEAVG:0.055 NUM:0.055 SynTraiAuc:0.481 RFAcc:0.962                   
[13000/24727] LG:1.130 LD:0.076 D:0.302 GP:0.014 RMSEAVG:0.055 NUM:0.055 SynTraiAuc:0.481 RFAcc:0.962                   
[13500/24727] LG:0.667 LD:0.280 

needed sample                                                                                                           
934                                                                                                                     
[    0/2360] LG:0.301 LD:-1.065 D:0.106 GP:0.073 RMSEAVG:0.277 NUM:0.277 SynTraiAuc:0.484 RFAcc:1.000                   
[  500/2360] LG:0.098 LD:1.543 D:1.684 GP:0.009 RMSEAVG:0.277 NUM:0.277 SynTraiAuc:0.484 RFAcc:1.000                    
[ 1000/2360] LG:0.242 LD:1.511 D:1.599 GP:0.006 RMSEAVG:0.277 NUM:0.277 SynTraiAuc:0.484 RFAcc:1.000                    
[ 1500/2360] LG:-0.188 LD:0.927 D:0.980 GP:0.003 RMSEAVG:0.077 NUM:0.077 SynTraiAuc:0.484 RFAcc:0.997                   
[ 2000/2360] LG:-0.108 LD:0.789 D:0.867 GP:0.005 RMSEAVG:0.077 NUM:0.077 SynTraiAuc:0.484 RFAcc:0.997                   
[ 2360/2360] LG:-0.161 LD:0.737 D:0.808 GP:0.004 RMSEAVG:0.077 NUM:0.077 SynTraiAuc:0.484 RFAcc:0.997                   
needed sample                   

[ 8000/13760] LG:0.493 LD:0.803 D:0.895 GP:0.005 RMSEAVG:0.030 NUM:0.030 SynTraiAuc:0.507 RFAcc:0.984                   
[ 8500/13760] LG:0.544 LD:0.780 D:0.925 GP:0.008 RMSEAVG:0.030 NUM:0.030 SynTraiAuc:0.507 RFAcc:0.984                   
[ 9000/13760] LG:0.538 LD:0.740 D:0.847 GP:0.006 RMSEAVG:0.037 NUM:0.037 SynTraiAuc:0.507 RFAcc:0.983                   
[ 9500/13760] LG:0.477 LD:0.799 D:0.897 GP:0.005 RMSEAVG:0.037 NUM:0.037 SynTraiAuc:0.507 RFAcc:0.983                   
[10000/13760] LG:0.458 LD:0.750 D:0.842 GP:0.005 RMSEAVG:0.029 NUM:0.029 SynTraiAuc:0.477 RFAcc:0.972                   
[10500/13760] LG:0.463 LD:0.812 D:0.938 GP:0.007 RMSEAVG:0.029 NUM:0.029 SynTraiAuc:0.477 RFAcc:0.972                   
[11000/13760] LG:0.453 LD:0.774 D:0.859 GP:0.004 RMSEAVG:0.029 NUM:0.029 SynTraiAuc:0.477 RFAcc:0.972                   
[11500/13760] LG:0.445 LD:0.604 D:0.694 GP:0.005 RMSEAVG:0.025 NUM:0.025 SynTraiAuc:0.477 RFAcc:0.970                   
[12000/13760] LG:0.422 LD:0.713 

[23000/25517] LG:0.673 LD:0.370 D:0.500 GP:0.009 RMSEAVG:0.026 NUM:0.026 SynTraiAuc:0.752 RFAcc:0.959                   
[23500/25517] LG:0.772 LD:0.391 D:0.489 GP:0.007 RMSEAVG:0.026 NUM:0.026 SynTraiAuc:0.752 RFAcc:0.959                   
[24000/25517] LG:0.729 LD:0.555 D:0.674 GP:0.009 RMSEAVG:0.036 NUM:0.036 SynTraiAuc:0.752 RFAcc:0.937                   
[24500/25517] LG:0.639 LD:0.422 D:0.521 GP:0.007 RMSEAVG:0.036 NUM:0.036 SynTraiAuc:0.752 RFAcc:0.937                   
[25000/25517] LG:0.618 LD:0.546 D:0.673 GP:0.009 RMSEAVG:0.027 NUM:0.027 SynTraiAuc:0.612 RFAcc:0.960                   
[25500/25517] LG:0.569 LD:0.583 D:0.655 GP:0.005 RMSEAVG:0.027 NUM:0.027 SynTraiAuc:0.612 RFAcc:0.960                   
[25517/25517] LG:0.548 LD:0.502 D:0.597 GP:0.007 RMSEAVG:0.027 NUM:0.027 SynTraiAuc:0.612 RFAcc:0.960                   
needed sample                                                                                                           
930                             

[ 2000/13680] LG:0.768 LD:1.007 D:1.078 GP:0.009 RMSEAVG:0.046 NUM:0.046 SynTraiAuc:0.789 RFAcc:0.965                   
[ 2500/13680] LG:0.817 LD:0.868 D:0.925 GP:0.007 RMSEAVG:0.038 NUM:0.038 SynTraiAuc:0.789 RFAcc:0.975                   
[ 3000/13680] LG:0.889 LD:0.957 D:1.040 GP:0.010 RMSEAVG:0.038 NUM:0.038 SynTraiAuc:0.789 RFAcc:0.975                   
[ 3500/13680] LG:0.960 LD:0.781 D:0.838 GP:0.007 RMSEAVG:0.038 NUM:0.038 SynTraiAuc:0.789 RFAcc:0.975                   
[ 4000/13680] LG:0.953 LD:0.636 D:0.680 GP:0.006 RMSEAVG:0.023 NUM:0.023 SynTraiAuc:0.789 RFAcc:0.967                   
[ 4500/13680] LG:1.001 LD:0.788 D:0.827 GP:0.005 RMSEAVG:0.023 NUM:0.023 SynTraiAuc:0.789 RFAcc:0.967                   
[ 5000/13680] LG:0.908 LD:0.676 D:0.828 GP:0.019 RMSEAVG:0.036 NUM:0.036 SynTraiAuc:0.743 RFAcc:0.959                   
[ 5500/13680] LG:0.975 LD:0.772 D:0.842 GP:0.009 RMSEAVG:0.036 NUM:0.036 SynTraiAuc:0.743 RFAcc:0.959                   
[ 6000/13680] LG:1.164 LD:0.704 

[ 5000/5360] LG:1.183 LD:0.434 D:0.689 GP:0.014 RMSEAVG:0.129 NUM:0.129 SynTraiAuc:0.657 RFAcc:0.991                    
[ 5360/5360] LG:1.200 LD:0.006 D:0.504 GP:0.028 RMSEAVG:0.129 NUM:0.129 SynTraiAuc:0.657 RFAcc:0.991                    
needed sample                                                                                                           
928                                                                                                                     
[    0/7320] LG:0.229 LD:-3.073 D:-0.071 GP:0.177 RMSEAVG:0.355 NUM:0.355 SynTraiAuc:0.522 RFAcc:1.000                  
[  500/7320] LG:1.275 LD:2.061 D:2.094 GP:0.002 RMSEAVG:0.355 NUM:0.355 SynTraiAuc:0.522 RFAcc:1.000                    
[ 1000/7320] LG:0.987 LD:1.567 D:1.609 GP:0.002 RMSEAVG:0.355 NUM:0.355 SynTraiAuc:0.522 RFAcc:1.000                    
[ 1500/7320] LG:1.055 LD:1.303 D:1.353 GP:0.003 RMSEAVG:0.067 NUM:0.067 SynTraiAuc:0.522 RFAcc:1.000                    
[ 2000/7320] LG:1.288 LD:1.408 D

In [4]:
write_file = f"Defect_Eclipse_PDE_UI_res_r3_rn249.csv"
write_path = f"{os.path.dirname(os.getcwd())}/result/Defect_Eclipse_PDE_UI/{write_file}"

data_path = f"{os.path.dirname(os.getcwd())}/data/imbalance_defects_prediction/7_CK_NET_PROC/input/Eclipse_PDE_UI--CK_NET_PROC.arff"
data = arff.loadarff(data_path)
df = pd.DataFrame(data[0])
df['isBug'] = df['isBug'].astype('str')
d = {'b\'YES\'': 1, 'b\'NO\'': 0}
df['isBug'] = df['isBug'].astype(str).map(d).fillna(df['isBug'])
df = df.drop_duplicates()
df.reset_index(inplace=True, drop=True)

X = df.iloc[:, :-1]
y = df.iloc[:, -1]
print("y value counts: \n", str(y.value_counts()))
print("y class ratio: 1:", str(round(y.value_counts()[0]/y.value_counts()[1])))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=249)
print("--- y train classes count: \n" + str(y_train.value_counts()))
print("--- y train ratio: 1:" + str(round(y_train.value_counts()[0] / y_train.value_counts()[1])))
print(" ")
print("--- y test classes count: \n" + str(y_test.value_counts()))
print("--- y test ratio: 1:" + str(round(y_test.value_counts()[0] / y_test.value_counts()[1])))

col = X_train.columns
tar = y_train.name
num_tuples_to_generate = int(y_train.value_counts()[0] - y_train.value_counts()[1])

X_train_SDV = X_train.copy()
y_train_SDV = y_train.copy()
X_train_SDV[tar] = y_train_SDV
pos_df = X_train_SDV[X_train_SDV[tar] == 1]
pos_df = pos_df.iloc[:, :-1]

metadata = SingleTableMetadata()
metadata.detect_from_dataframe(data=pos_df)

start_time = time.time()
syn3 = CTGANSynthesizer(metadata)
syn3.fit(data=pos_df)
X_train_new = syn3.sample(num_rows=num_tuples_to_generate).to_numpy()

rt = time.time() - start_time

X_train_new = pd.DataFrame(np.vstack((X_train.to_numpy(), X_train_new)), columns=col)
y_train_new = np.ones(num_tuples_to_generate)
y_train_new = pd.Series(np.hstack((y_train.to_numpy(), y_train_new)), name=tar)

# scale data
scaler = StandardScaler()
X_train_scale = pd.DataFrame(scaler.fit_transform(X_train_new), columns=X_train_new.columns, index=X_train_new.index)
X_test_scale = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

print("y train ratio: 1:" + str(round(y_train_new.value_counts()[0] / y_train_new.value_counts()[1])))

# create models
clf_SVM = SVC()
clf_KNN = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
clf_LR = LogisticRegression(random_state=42, solver="saga", max_iter=20000, n_jobs=-1)
clf_DT = DecisionTreeClassifier()
clf_RF = RandomForestClassifier(random_state=42, n_jobs=-1)
clf_LightGBM = LGBMClassifier(objective="binary", random_state=42, n_jobs=-1)
clf_Adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
clf_GBDT = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)

clf_SVM.fit(X_train_scale, y_train_new)
clf_KNN.fit(X_train_scale, y_train_new)
clf_LR.fit(X_train_scale, y_train_new)
clf_DT.fit(X_train_scale, y_train_new)
clf_RF.fit(X_train_scale, y_train_new)
clf_LightGBM.fit(X_train_scale, y_train_new)
clf_Adaboost.fit(X_train_scale, y_train_new)
clf_GBDT.fit(X_train_scale, y_train_new)

y_pred_SVM = clf_SVM.predict(X_test_scale)
y_pred_KNN = clf_KNN.predict(X_test_scale)
y_pred_LR = clf_LR.predict(X_test_scale)
y_pred_DT = clf_DT.predict(X_test_scale)
y_pred_RF = clf_RF.predict(X_test_scale)
y_pred_LightGBM = clf_LightGBM.predict(X_test_scale)
y_pred_Adaboost = clf_Adaboost.predict(X_test_scale)
y_pred_GBDT = clf_GBDT.predict(X_test_scale)

with open(write_path, "a", newline="") as f:
    csv_writer = csv.writer(f)

    csv_writer.writerow(["SDV_GAN", rt, "SVM"] + evaluate_result(y_pred_SVM, y_test))
    csv_writer.writerow(["SDV_GAN", rt, "KNN"] + evaluate_result(y_pred_KNN, y_test))
    csv_writer.writerow(["SDV_GAN", rt, "LR"] + evaluate_result(y_pred_LR, y_test))
    csv_writer.writerow(["SDV_GAN", rt, "DT"] + evaluate_result(y_pred_DT, y_test))
    csv_writer.writerow(["SDV_GAN", rt, "RF"] + evaluate_result(y_pred_RF, y_test))
    csv_writer.writerow(["SDV_GAN", rt, "LightGBM"] + evaluate_result(y_pred_LightGBM, y_test))
    csv_writer.writerow(["SDV_GAN", rt, "Adaboost"] + evaluate_result(y_pred_Adaboost, y_test))
    csv_writer.writerow(["SDV_GAN", rt, "GBDT"] + evaluate_result(y_pred_GBDT, y_test))

y value counts: 
 0    1288
1     209
Name: isBug, dtype: int64
y class ratio: 1: 6
--- y train classes count: 
0    1030
1     167
Name: isBug, dtype: int64
--- y train ratio: 1:6
 
--- y test classes count: 
0    258
1     42
Name: isBug, dtype: int64
--- y test ratio: 1:6


/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/numerical.py:112: UserWarning: No rounding scheme detected for column 'BonPwr'. Data will not be rounded.
  warnings.warn(
/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/numerical.py:112: UserWarning: No rounding scheme detected for column 'Between'. Data will not be rounded.
  warnings.warn(
/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/home/lingxiao/diveplane/lib/python3.8/site-packa

/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missi

/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/home/lingxiao/diveplane/lib/python3.8/site-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missi

y train ratio: 1:1


In [9]:
data_path = f"{os.path.dirname(os.getcwd())}/data/imbalance_defects_prediction/1_CK/input/ant-1.4--CK.arff"
data = arff.loadarff(data_path)
df = pd.DataFrame(data[0])
df['isBug'] = df['isBug'].astype('str')
d = {'b\'YES\'': 1, 'b\'NO\'': 0}
df['isBug'] = df['isBug'].astype(str).map(d).fillna(df['isBug'])
df = df.drop_duplicates()
df.reset_index(inplace=True, drop=True)

X = df.iloc[:, :-1]
y = df.iloc[:, -1]
print("y value counts: \n", str(y.value_counts()))
print("y class ratio: 1:", str(round(y.value_counts()[0]/y.value_counts()[1])))

y value counts: 
 0    138
1     39
Name: isBug, dtype: int64
y class ratio: 1: 4
